So I guess we want to have the JSON schema use mixedCase, and the emitted JSON use mixedCase, but the clip attributes should be snake_case -- and not necessarily the snakeification of the mixedCase.

In [117]:
from typing import Any, List
from fractions import Fraction
import numbers
import json

from pydantic import BaseModel
from camdkit.backwards import CompatibleBaseModel
from camdkit.numeric_types import StrictlyPositiveRational



def test2():
    # takes_rational_two(1)
    # # takes_rational_two(1.3)
    # takes_rational_two(Fraction(24000, 1001))
    # # takes_rational_two(2 + 0j)
    for thing in (1, 1.3, StrictlyPositiveRational(24000,1001), Fraction(30000, 1001), 1+2j, 'foo'):
        try:
            thing = rationalize(thing)
            print(f"type of {thing} is {type(thing)}")
        except ValueError as e:
            print(e)


test2()

type of num=1 denom=1 is <class 'camdkit.numeric_types.StrictlyPositiveRational'>
could not promote input of type <class 'float'> to a StrictlyPositiveRational
type of num=24000 denom=1001 is <class 'camdkit.numeric_types.StrictlyPositiveRational'>
type of num=30000 denom=1001 is <class 'camdkit.numeric_types.StrictlyPositiveRational'>
could not promote input of type <class 'complex'> to a StrictlyPositiveRational
could not promote input of type <class 'str'> to a StrictlyPositiveRational


In [119]:
class Position(CompatibleBaseModel):
    x: int
    y: int


class PositionTracker(CompatibleBaseModel):
    positions: tuple[Position, ...]

class Cleep(CompatibleBaseModel):
    tracker: PositionTracker

tracker = PositionTracker(positions=(Position(x=1, y=2), Position(x=2, y=3)))
cleep = Cleep(tracker=tracker)

print(json.dumps(cleep.model_dump(), indent=2))


{
  "tracker": {
    "positions": [
      {
        "x": 1,
        "y": 2
      },
      {
        "x": 2,
        "y": 3
      }
    ]
  }
}


In [3]:
import json
foo = {'type': 'object',
 'properties': {'radial': {'type': 'array', 'items': {'type': 'number'}},
  'tangential': {'anyOf': [{'type': 'array', 'items': {'type': 'number'}},
    {'type': 'null'}],
   'default': None},
  'model': {'anyOf': [{'type': 'string', 'minLength': 1, 'maxLength': 1023},
    {'type': 'null'}],
   'default': None}},
 'required': ['radial'],
 'additionalProperties': False,
 '$defs': {'NonBlankUTF8String': {'type': 'string',
   'minLength': 1,
   'maxLength': 1023}}}
print(json.dumps(foo, indent=2))

{
  "type": "object",
  "properties": {
    "radial": {
      "type": "array",
      "items": {
        "type": "number"
      }
    },
    "tangential": {
      "anyOf": [
        {
          "type": "array",
          "items": {
            "type": "number"
          }
        },
        {
          "type": "null"
        }
      ],
      "default": null
    },
    "model": {
      "anyOf": [
        {
          "type": "string",
          "minLength": 1,
          "maxLength": 1023
        },
        {
          "type": "null"
        }
      ],
      "default": null
    }
  },
  "required": [
    "radial"
  ],
  "additionalProperties": false,
  "$defs": {
    "NonBlankUTF8String": {
      "type": "string",
      "minLength": 1,
      "maxLength": 1023
    }
  }
}
